In [116]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import requests
import pandas as pd
import numpy as np
import json
import time
import re

from dotenv import load_dotenv
import os

load_dotenv()

True

In [117]:
# Função que substitui multiplas quebras de linha por uma única
def remove_multiple_breaks(text):
    return re.sub(r'\n+', '\n', text)

# Função que remove espaços em branco no início e no final de uma string
def remove_whitespace(text):
    return re.sub(r'^\s+|\s+$', '', text)

# Função que remove múltiplos espaços em branco
def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text)

# Função que substitui as quebras de linha por espaços em branco
def remove_breaks(text):
    return re.sub(r'\n', ' ', text)

# Função que remove "About the job" e "See less"
def clean_description(text):
    return re.sub(r'About the job|See less', '', text)

# Função que limpa o link da vaga para seguir a formatação https://www.linkedin.com/jobs/view/id/
def clean_link(url):
    return '/'.join(url.split('/')[:6]) + '/'

In [118]:
job_url = input('Link da vaga: ')

options = Options()
#options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

driver.get('https://www.linkedin.com/login')

email = os.getenv('LINKEDIN_EMAIL')
password = os.getenv('LINKEDIN_PASSWORD')

driver.find_element(By.ID, 'username').send_keys(email)
driver.find_element(By.ID, 'password').send_keys(password)
driver.find_element(By.CSS_SELECTOR, '.login__form_action_container button').click()

#job_url = 'https://www.linkedin.com/jobs/view/3902888840/'
#job_url = 'https://www.linkedin.com/jobs/view/3878872267/' # inChurch
driver.get(job_url)

# ---------------------------------------------------------------------------------------------------------------------------------------

see_more_button_id = 'ember41'
see_more_button = driver.find_element(By.ID, see_more_button_id).click()

# Scraper
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Título da vaga
titulo = soup.find('h1', {'class': 't-24 t-bold job-details-jobs-unified-top-card__job-title'}).text.strip() if soup.find('h1', {'class': 't-24 t-bold job-details-jobs-unified-top-card__job-title'}) else 'Not found'

time.sleep(1)

# Nome da empresa, Local, Data de postagem, Número de aplicantes
infos = soup.find('div', {'class': 'job-details-jobs-unified-top-card__primary-description-without-tagline mb2'}).text.strip() if soup.find('div', {'class': 'job-details-jobs-unified-top-card__primary-description-without-tagline mb2'}) else 'Not found'
if infos != 'Not found':
    empresa, local, data_postagem, num_aplicantes = infos.split(' · ')
else:
    empresa = local = data_postagem = num_aplicantes = 'Not found'

# Modalidade de contratação, Nível de experiência
infos2 = soup.find('li', {'class': 'job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight'}).text.strip() if soup.find('li', {'class': 'job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight'}) else 'Not found'
if infos2 != 'Not found':
    modalidade, nivel = remove_multiple_spaces(remove_multiple_breaks(infos2)).split(' ')[:2]
else:
    modalidade = nivel = 'Not found'

# Número de funcionários, Área da empresa
infos3 = soup.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})
if len(infos3) > 1 and infos3[1].text.strip() != 'Not found':
    infos3 = infos3[1].text.strip()
    if ' · ' in infos3:
        num_funcionarios, area = infos3.split(' · ')
    else:
        num_funcionarios = infos3
else:
    num_funcionarios = area = 'Not found'

# Descrição da vaga
descricao = soup.find_all('div', {'class': 'mt4'})
if len(descricao) > 1 and descricao[1].text.strip() != 'Not found':
    descricao = descricao[1].text.strip()
    descricao = remove_multiple_breaks(descricao)
    descricao = remove_multiple_spaces(descricao)
    descricao = clean_description(descricao)
else:
    descricao = 'Not found'

driver.quit()

# Printar as informações
print(f'Título da vaga: {titulo}')

print(f'Empresa: {empresa}')
print(f'Local: {local}')
print(f'Data de postagem: {data_postagem}')
print(f'Número de aplicantes: {num_aplicantes}')

print()

print(f'Modalidade: {modalidade}')
print(f'Nível de experiência: {nivel}')

print()

print(f'Número de funcionários: {num_funcionarios}')
print(f'Área da empresa: {area}')
print(f'Descrição da vaga: {descricao}')


Título da vaga: Estágio em Desenvolvimento de Software
Empresa: Nomus
Local: Brasil
Data de postagem: há 2 meses
Número de aplicantes: Mais de 100 candidaturas

Modalidade: Remoto
Nível de experiência: Tempo

Número de funcionários: 51-200 funcionários
Área da empresa: Atividades dos serviços de tecnologia da informação
Descrição da vaga: Sobre a vaga A área de “Desenvolvimento” é responsável pelo desenvolvimento de softwares de gestão incríveis.Seus principais objetivos são desenvolver soluções que geram resultado para nossos clientes, com velocidade, qualidade e facilidade de uso.Como estagiário em Desenvolvimento você trabalhará em conjunto com nosso time desenvolvendo novas funcionalidades nos softwares de gestão da Nomus.Essa é uma oportunidade incrível de aprendizado, você irá aprender muito sobre Java, Javascript, struts, Spring, hibernate, jQuery e css, e conseguirá enxergar os resultados do seu trabalho na prática.O valor da bolsa inicial para a carga horária de 30 horas por s

In [119]:
AWAN_API_KEY = os.getenv('AWAN_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

AWAN_URL = 'https://api.awanllm.com/v1/chat/completions'

In [120]:
print(f'Aproximadamente {len(descricao.split(" "))} tokens')

Aproximadamente 419 tokens


In [121]:
prompt = f'''Dada a descrição de uma vaga: {descricao}

Preencha esses campos:

(Sua resposta deve incluir apenas e somente o que foi pedido, seguindo a exata formatação dos campos)

*Data de formação*:
*Salário/Bolsa-Auxílio/Bolsa-Estágio*:
*Benefícios*: (Cada benefício deve estar em um bullet point)
	- 
	- 
	- 
 
Caso não haja informação sobre quando deve ser o período de formação, salário ou benefícios, preencha somente com N/A.
Não incluir trabalho remoto/home office nem salário/bolsa estágio/bolsa auxílio como benefício. 
Responda em português brasileiro.
'''

In [122]:
prompt2 = f'''Preencha os campos abaixo (seguindo a formatação exata) de acordo com a descrição da vaga fornecida: {descricao}

*Data de formação*:
*Salário/Bolsa*:

*Benefícios*:
    - 
    - 
    - 

Respeite os campos e não inclua informações que não foram solicitadas.
Caso não haja informação sobre algum campo, preencha com N/A. Não inclua trabalho remoto ou salário/bolsa como benefícios. 
'''

In [123]:
payload = json.dumps({
    "model": "Meta-Llama-3-8B-Instruct-Dolfin-v0.1",
    "messages": [
        {
            "role": "user",
            "content": prompt2
        }
    ]
})

headers = {
    'Content-Type': 'application/json',
    'Authorization': f"Bearer {AWAN_API_KEY}"
}

In [124]:
response = requests.request('POST', AWAN_URL, headers=headers, data=payload)

In [125]:
response.json()

{'id': 'cmpl-e7794abc30bb480484378bf2a66f33cb',
 'object': 'chat.completion',
 'created': 161866,
 'model': 'Meta-Llama-3-8B-Instruct-Dolfin',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '*Data de formação*: N/A\n*Salário/Bolsa*: R$ 1.800,00 a R$ 3.600,00\n*Benefícios*:\n    - Seguro de vida em grupo\n    - Cartão flexível Caju\n    - Plano de saúde\n    - Plano odontológico\n    - Auxílio home-office\n    - Auxílio educação\n    - Bônus semestral alinhado com a avaliação de desempenho e resultado da Nomus\n    - Plano de carreira\n    - Horário flexível\n    - Home-office\n    - Equipamentos fornecidos pela empresa\n    - Gympass\n    - Day off\n    - Licença maternidade e paternidade estendida\n    - Auxílio-coworking\n    - Auxílio-creche para filhos'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 963,
  'total_tokens': 1157,
  'completion_tokens': 194}}

In [126]:
print(response.json()['choices'][0]['message']['content'])

*Data de formação*: N/A
*Salário/Bolsa*: R$ 1.800,00 a R$ 3.600,00
*Benefícios*:
    - Seguro de vida em grupo
    - Cartão flexível Caju
    - Plano de saúde
    - Plano odontológico
    - Auxílio home-office
    - Auxílio educação
    - Bônus semestral alinhado com a avaliação de desempenho e resultado da Nomus
    - Plano de carreira
    - Horário flexível
    - Home-office
    - Equipamentos fornecidos pela empresa
    - Gympass
    - Day off
    - Licença maternidade e paternidade estendida
    - Auxílio-coworking
    - Auxílio-creche para filhos


In [127]:
descricao_llm = response.json()['choices'][0]['message']['content']

In [128]:
msg = f'''*[{modalidade}] {titulo}*
_*{empresa}* - {local} - {num_funcionarios}_

{descricao_llm}

*Postado*: {data_postagem} 
*Aplicantes*: {num_aplicantes}

*Link da vaga*: {clean_link(job_url)}
'''

In [129]:
print(msg)

*[Remoto] Estágio em Desenvolvimento de Software*
_*Nomus* - Brasil - 51-200 funcionários_

*Data de formação*: N/A
*Salário/Bolsa*: R$ 1.800,00 a R$ 3.600,00
*Benefícios*:
    - Seguro de vida em grupo
    - Cartão flexível Caju
    - Plano de saúde
    - Plano odontológico
    - Auxílio home-office
    - Auxílio educação
    - Bônus semestral alinhado com a avaliação de desempenho e resultado da Nomus
    - Plano de carreira
    - Horário flexível
    - Home-office
    - Equipamentos fornecidos pela empresa
    - Gympass
    - Day off
    - Licença maternidade e paternidade estendida
    - Auxílio-coworking
    - Auxílio-creche para filhos

*Postado*: há 2 meses 
*Aplicantes*: Mais de 100 candidaturas

*Link da vaga*: https://www.linkedin.com/jobs/view/3824128427/



In [130]:
from twilio.rest import Client
from twilio.twiml.messaging_response import MessagingResponse
from flask import Flask, request

account_sid = os.getenv('TWILIO_ACCOUNT_SID')
auth_token = os.getenv('TWILIO_AUTH_TOKEN')
client = Client(account_sid, auth_token)

# app = Flask(__name__)

# @app.route("/webhook", methods=["POST"])
# def webhook():
#     # Captura a mensagem recebida
#     incoming_message = request.values.get("Body", "").lower()
#     # Neste exemplo, apenas exibimos a mensagem recebida
#     print("Mensagem recebida:", incoming_message)
    
#     # Responde à mensagem
#     response = MessagingResponse()
#     response.message("Obrigado por sua mensagem! Estamos processando.")
    
#     return str(response)

# Enviar mensagem do sandbox para +5512996039511
message = client.messages.create(
    body=msg,
    from_='whatsapp:+14155238886',
    to='whatsapp:+5512996039511'
)

print(message.sid)

SMe025e40e775f2c01ff2789a23f977568
